In [1]:
# Using Keras functional model
from keras.models import Model
from keras.layers import Input, Dense, LSTM
# Used for feeding data in Input 
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size =64 # Batchsize for network
epochs = 100 # Number of epochs
latent_dim = 256 # Number of LSTM cells
num_samples = 2000 # Number of samples

In [3]:
input_texts = [] # List of Input(English sentences)
target_texts = [] # List of Target(Hindi sentences)

In [4]:
input_chars = [] # List of unique english characters
target_chars = [] # List of unique hindi characters

In [61]:
file_path = '/Users/arth/Desktop/nmt-hindi-english/hin-eng/eng-hin.txt' # Path to english hindi text file

In [62]:
# reading file into lines
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [65]:
for line in lines[:min(num_samples, len(lines)-1)]:
    input_text, target_text = line.split('\t') # Splitting using tabspaces
    target_text = '\t' + target_text + '\n' # Using \n token to signify EOS 
    input_texts.append(input_text)
    target_texts.append(target_text)
    # Used to get unique characters
    for char in input_text:
        if char not in input_chars:
            input_chars.append(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.append(char)

In [66]:
input_chars = sorted(input_chars)
target_chars = sorted(target_chars) 

In [67]:
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(seq) for seq in input_texts]) # Maximum sequence length for english sentences
max_decoder_seq_length = max([len(seq) for seq in target_texts]) # Maximum sequence length for hindi sentences
max_encoder_seq_length,max_decoder_seq_length

(124, 123)

In [68]:
# Mapping character tokens to indices
input_token_index = dict( [(char, i) for i,char in enumerate(input_chars)])
target_token_index = dict( [(char, i) for i,char in enumerate(target_chars)])

In [69]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype= 'float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32')

In [70]:
# One-hot encoding input
for i, input_text in enumerate(input_texts):
    for j, ch in enumerate(input_text):
        encoder_input_data[i,j,input_token_index[ch]] =1

In [71]:
# One-hot encoding output
for i, target_text in enumerate(target_texts):
    for j, ch in enumerate(target_text):
        decoder_input_data[i,j,target_token_index[ch]] =1
        if j>0:
            decoder_target_data[i, j-1, target_token_index[ch]]=1

In [74]:
# Defining Encoder model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [75]:
encoder_states = [state_h, state_c]

In [76]:
# Defining Decoder model
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_state = True, return_sequences = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [77]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [78]:
model.compile(optimizer = 'rmsprop', loss= 'categorical_crossentropy')

In [81]:
history = model.fit([encoder_input_data,decoder_input_data], decoder_target_data, batch_size= batch_size, epochs=1, validation_split = 0.2)

Train on 7093 samples, validate on 1774 samples
Epoch 1/1
7093/7093 [==============================] - 144s 20ms/step - loss: 0.1089 - val_loss: 0.0825


In [82]:
model.save('eng-hin.h5')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [83]:
# Defining inference model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [84]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [96]:
def make_inference(sentence):
    input_data= np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype= 'float32')
    for j, ch in enumerate(sentence):
        input_data[1,j,input_token_index[ch]] =1
    decoded_sentence = decode_sequence(input_data)
    print('Input sentence:', sentence)
    print('Translated sentence:', decoded_sentence)

In [97]:
make_inference("Hello, How are you?")

Input sentence: Hello, How are you?
Translated sentence: मुझे यहाँ पर अजनबी के जैसा महसूस है।



In [98]:
make_inference("a")

Input sentence: a
Translated sentence: मुझे यहाँ पर अजनबी के जैसा महसूस है।



In [100]:
for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Wow!
Decoded sentence: मेरे पापा को गर्मी के मौसम से नफ़रत है।

-
Input sentence: Help!
Decoded sentence: हमने उसका हिसाब कुछ लीको का से हो।

-
Input sentence: Jump.
Decoded sentence: वह किताब कहाँ है?

-
Input sentence: Jump.
Decoded sentence: वह किताब कहाँ है?

-
Input sentence: Jump.
Decoded sentence: वह किताब कहाँ है?

-
Input sentence: Hello!
Decoded sentence: हमने उसका हिसाब कुछ लीको का से हो।

-
Input sentence: Hello!
Decoded sentence: हमने उसका हिसाब कुछ लीको का से हो।

-
Input sentence: Cheers!
Decoded sentence: हमने उसका जन्मदिन मनाया।

-
Input sentence: Cheers!
Decoded sentence: हमने उसका जन्मदिन मनाया।

-
Input sentence: Got it?
Decoded sentence: वह मेरे लिए बान नहीं पर्चास्स तुराने वादे मदं।

-
Input sentence: I'm OK.
Decoded sentence: मैं तुम्हें एक बात की सलाह देता हूँ।

-
Input sentence: Awesome!
Decoded sentence: मेरे पापा को गर्मी के मौसम से नफ़रत है।

-
Input sentence: Come in.
Decoded sentence: मुझे यह काता से मौलिया था।

-
Input sentence: Get out!